# Подготовка данных

<hr>
<strong>Требования к предобработк данных:</strong>

- убрать конфиденциальную информацию
- анонимизировать идентификаторы, использующиеся в системе
- нормализовать данные методом min_max 
- определить координаты населенных пунктов

<strong>Предобработка таблиц:<strong>

1. [Актуальных магазинов](#actual_market)
2. [Продажи по ассортиментам](#assortiment_market)
3. [Рознично-торговый оборот](#rto)
4. [Продажи алкоголя](#alcohol)

<a id='actual_market'></a>
### Обработка Актуальных магазинов

In [1]:
import pandas as pd
from secret_info import API_KEY, MIN, MAX
from tqdm import tqdm
from dateutil import parser
from datetime import datetime
import uuid, hashlib
import requests
from scripts import Id

Создаем класс, который будет хранить словарь идентификаторов в виде словаря
Он будет использоваться для  замены sap (id).

<code>Id.uuid_dict[454] = '65684580-e0f0-68b0-b407-89a4f9f6b6f8'</code>

In [2]:
new_id = Id()

In [3]:
df = pd.read_csv('data/1.actual_markets.csv')
del df['num_market']

new_id.get_uuid(df['sap']) # добавили в словарь новые id с конвертирование в UUID
df['sap'] = df['sap'].map(new_id.uuid_dict)
new_id.show()

Количество id:  1586


In [4]:
def str_to_bool(value: str) -> str:
    """да / нет ->  1 / 0"""
    if isinstance(value, str):
        value = value.strip().lower()
    if value == 'да' or value == 1:
        return 1
    elif value == 'нет' or value == 0:
        return 0
    
def minmax(x_array, x_min = None, x_max = None):
    """minmax стандартизация"""
    if any([not x_min, not x_max]):
        x_min = x_array.min()
        x_max = x_array.max()
    
    new_array = (x_array - x_min)/(x_max - x_min)
    return new_array

Нормализуем признаки 

In [5]:
# bool
columns = ['night_acceptance','night_display','store_concept','dominators']
for col_name in columns:
    df[col_name] = df[col_name].map(str_to_bool)
    
# numeric
columns = ['employee','cashier','area','cash_desks','tobacco_shop','universal_assortment']
for col_name in columns:
    df[col_name] = minmax(df[col_name])

#### Нахождение геолокации

У Yandex.Geocoder есть ограничение на кол-во запросов в сутки: не более 1000 запросов.
Возьмем уникальные значения и сопоставим через словарь из столбца <code>df.locality</code>, чтобы не повторяться и зря не расходовать трафик. И найдем все интересующие адреса.

In [6]:
address = df.locality.unique()
print(f'Уникальных адресов {len(address)} :\n',*address[:5],'... и т.д',sep='\n' )

Уникальных адресов 360 :

Санкт-Петербург
Коммунар
Тосно
Кировск
Сестрорецк
... и т.д


In [7]:
def get_coordinates(address_localtion : str):
    """Возвращает координаты адреса"""
    url = 'https://geocode-maps.yandex.ru/1.x/'
    latitude, longitude = None, None
    
    params = {
                'geocode': address_localtion,
                'format': 'json',
                'apikey': API_KEY  
              }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        coordinates = data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']
        longitude, latitude = map(float, coordinates.split())
        return latitude, longitude
    except requests.exceptions.RequestException as e:
        
        print(f"An error occurred: {e}")
        pass # TODO написать логирвание ошибок


In [8]:
coordinates = [get_coordinates(name) for name in tqdm(address)]

100%|████████████████████████████████████████████████████████████████████████████████| 360/360 [01:02<00:00,  5.74it/s]


In [9]:
location = dict(zip(address, coordinates))
try:
    df.insert(5,'coordinate',df.locality.map(location))  
except ValueError as ex:
    print("df['coordinate'] уже добавили")

In [10]:
df.head(3)

,sap,employee,cashier,locality,area,coordinate,night_acceptance,night_display,cash_desks,store_concept,tobacco_shop,dominators,universal_assortment
0,c7cfd542-9765-bd92-0240-ecb7fcd8452e,0.326531,0.342857,Санкт-Петербург,0.338849,"(59.938955, 30.315644)",1,1,0.357143,1,0.454545,1,0.523810
1,35373725-79f5-c17d-8c63-d347d20bf0a1,0.265306,0.057143,Коммунар,0.320144,"(59.621609, 30.393483)",1,0,0.214286,1,0.181818,1,0.285714
2,35e29d3a-bdca-a913-d54d-5da0fb69aa29,0.734694,0.200000,Санкт-Петербург,0.722302,"(59.938955, 30.315644)",1,1,0.500000,1,0.090909,1,0.761905


In [11]:
df.to_csv('data/preprocessed/actual__preprocessed.csv',index = False)
del df 

<a id='assortiment_market'></a>

### Обработка продаж по ассортиментам

In [12]:
df = pd.read_csv('data/2.type_meal_sells.csv')
del df['Магазин']

new_id.get_uuid(df['sap'])
df['sap'] = df['sap'].map(new_id.uuid_dict)

<code>dateutil.parser</code> хорошо работает на английском. Он используется для автоматического определения времени и даты <code>datetime</code> независимо от формата.

In [13]:
months_rus = ['НОЯ', 'ДЕК','ЯНВ', 'ФЕВ', 'МАР', 'АПР',
              'МАЙ', 'ИЮН','ИЮЛ', 'АВГ', 'СЕН', 'ОКТ']
months_eng = ['NOV','DEC','JAN','FEB','MAR','APR',
              'MAY','JUN','JUL','AUG','SEP','OCT',]

month = dict(zip(months_rus,months_eng))
new_months = [parser.parse(month[m[:3]] + m[3:], dayfirst=False).strftime('%Y-%m') for m in df.columns[3:]]

In [14]:
new_columns = ['sap','type_meal'] + new_months
df = df.rename(columns=dict(zip(df.columns,new_columns)))

Нормализуем значения временных рядов

In [15]:
for col_name in df.columns[2:]:
    df[col_name] = minmax(df[col_name], x_min = MIN, x_max = MAX)

In [16]:
df.head(3)

,sap,type_meal,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,...,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,ИЮЛ 2021
0,045af141-72d1-3163-f9ef-cea8af3547fb,Овощи/Грибы,0.092780,0.089454,0.098488,0.100432,0.137098,0.116499,0.108158,0.116955,...,0.078114,0.103932,0.107777,0.107881,0.117877,0.105708,0.108041,0.132824,0.132424,0.109873
1,045af141-72d1-3163-f9ef-cea8af3547fb,Овощи/Зелень,0.058501,0.062190,0.068631,0.067251,0.095278,0.090512,0.068993,0.075536,...,0.060752,0.073966,0.077267,0.085635,0.083643,0.089196,0.086589,0.079528,0.077316,0.107070
2,045af141-72d1-3163-f9ef-cea8af3547fb,Фрукты/Экзотика,0.090461,0.115416,0.093577,0.087794,0.090266,0.084540,0.054202,0.051428,...,0.123896,0.111636,0.124712,0.110958,0.094535,0.075565,0.057278,0.043437,0.043367,0.085383


In [17]:
df.to_csv('data/preprocessed/type_meal__preprocessed.csv', index = False)
del df

<a id='rto'></a>
### Обработка рознично-торгового оборота

In [18]:
df = pd.read_csv('data/3.retail_turnover.csv')
del df['Упр. Кластер (НА)']
del df['Дата открытия']
del df['Дата закрытия|Календарный день']


In [19]:
df = df.rename(columns={'ЦФО (НА)':'sap'})
df['sap'] = df['sap'].map(lambda x :x[4:])

new_id.get_uuid(df['sap'])
df['sap'] = df['sap'].map(new_id.uuid_dict)

Нормализуем значения временных рядов

In [20]:
for col_name in df.columns[1:]:
    df[col_name] = minmax(df[col_name], x_min = MIN, x_max = MAX)

In [21]:
df.head(2)

,sap,01.07.2018,02.07.2018,03.07.2018,04.07.2018,05.07.2018,06.07.2018,07.07.2018,08.07.2018,09.07.2018,...,21.06.2021,22.06.2021,23.06.2021,24.06.2021,25.06.2021,26.06.2021,27.06.2021,28.06.2021,29.06.2021,30.06.2021
0,c3d9a2f4-fd5a-234f-cf06-daa84cb126cb,0.258308,0.288219,0.308172,0.258617,0.302047,0.286274,0.262396,0.248024,0.267803,...,0.218839,0.217433,0.249612,0.249087,0.221760,0.143271,0.225221,0.246808,0.233346,0.297213
1,35d47589-95e2-d4b8-92bf-fb11011bf564,0.318323,0.219351,0.241947,0.239500,0.273389,0.255926,0.295464,0.365969,0.212947,...,0.267301,0.276723,0.288704,0.295780,0.273941,0.232721,0.326351,0.287766,0.301064,0.316988


In [22]:
df.to_csv('data/preprocessed/retail_turnover__preprocessed.csv',index = False)
del df 

<a id='alcohol'></a>
### Продажа алкоголя

In [23]:
df = pd.read_csv('data/4.alcohol_sells.csv')
del df['Упр. Кластер (НА)']

In [24]:
new_months = [parser.parse(month[m[:3]] + m[3:], dayfirst=False).strftime('%Y-%m') for m in df.columns[1:]]
new_columns = ['sap'] + new_months
df = df.rename(columns=dict(zip(df.columns,new_columns)))

new_id.get_uuid(df['sap'])
df['sap'] = df['sap'].map(new_id.uuid_dict)

In [25]:
new_id.show()

Количество id:  2851


Нормализуем значения временных рядов

In [26]:
for col_name in df.columns[1:]:
    df[col_name] = minmax(df[col_name],x_min = MIN, x_max = MAX)

In [27]:
df.head(2)

,sap,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,...,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07
0,045af141-72d1-3163-f9ef-cea8af3547fb,0.175942,0.165586,0.168935,0.161475,0.193564,0.176264,0.154633,0.162216,0.173108,...,0.171533,0.174348,0.185261,0.190989,0.191204,0.179692,0.181616,0.144125,0.127334,0.120985
1,587a9668-3c38-455d-e5fb-efd5ce206d07,0.098581,0.103458,0.118818,0.116952,0.133351,0.138954,0.147771,0.137995,0.142700,...,0.098157,0.087995,0.076986,0.087191,0.084932,0.084825,0.081750,0.079863,0.075408,0.066010


In [28]:
df.to_csv('data/preprocessed/aclohol__preprocessed.csv',index = False)
del df